In [ ]:
#Author: abhishek.sawalkar@rutgers.edu
#Script: To find the common genes across different subtypes and then map it to diff gene expression log2fold change and padj values
#env : env_ml

In [2]:
#Import libraries
import pandas as pd

In [10]:
result_path = './results/co-op_learning/results_subtype_vs_control_consistency_final'
et_file = result_path + "/ET_FeatureCounts.csv"
pv_file = result_path + "/PV_FeatureCounts.csv"
mf_file = result_path + "/mf_FeatureCounts.csv"

et_data = pd.read_csv(et_file)
pv_data = pd.read_csv(pv_file)
mf_data = pd.read_csv(mf_file)

et_genes = set(et_data.iloc[:,0])
pv_genes = set(pv_data.iloc[:,0])
mf_genes = set(mf_data.iloc[:,0])
common_genes = et_genes & pv_genes & mf_genes

In [50]:
et_out = result_path + "/ET_Gene_Expression.csv"
pv_out = result_path + "/PV_Gene_Expression.csv"
mf_out = result_path + "/MF_Gene_Expression.csv"

In [25]:
diff_path = './data/diff_gene_data/'
et_diff = diff_path + 'ET_CTRL.csv'
pv_diff = diff_path + 'PV_CTRL.csv'
mf_diff = diff_path + 'MF_CTRL.csv'
e
et_diff = pd.read_csv(et_diff)
pv_diff = pd.read_csv(pv_diff)
mf_diff = pd.read_csv(mf_diff)


In [28]:
print(et_diff.head(1))

   Unnamed: 0                gene   baseMean  log2FoldChange     lfcSE  \
0        2907  ENSG00000107295.10  26.885494        0.953952  0.470081   

        stat        pvalue          padj     ensg.cleaned hgnc_symbol  
0  11.900516  1.176174e-32  2.626044e-28  ENSG00000107295      SH3GL2  


In [39]:
len(common_genes)

23

In [52]:
# Remove prefix (e.g., X_ or Z_) from common_genes for comparison
common_genes_cleaned = {gene.split('_', 1)[-1] for gene in common_genes}

# Filter data: Match hgnc_symbol with cleaned common_genes
final_table = et_diff[et_diff['hgnc_symbol'].isin(common_genes_cleaned)].copy()

# Add the prefix back to matched genes
final_table['Gene'] = final_table['hgnc_symbol'].apply(
    lambda symbol: next((gene for gene in common_genes if gene.endswith(symbol)), None)
)

# Reorganize columns to have prefixed_gene as the first column
columns_order = ['Gene','log2FoldChange','padj','pvalue','baseMean','lfcSE','stat'] #+ [col for col in final_table.columns if col != 'prefixed_gene']
final_table = final_table[columns_order]
# Sort the final table by padj
final_table = final_table.sort_values(by='padj', ascending=True)

In [53]:
final_table.to_csv(et_out,index=False)

In [55]:

# Filter data: Match hgnc_symbol with cleaned common_genes
pv_final_table = pv_diff[pv_diff['hgnc_symbol'].isin(common_genes_cleaned)].copy()

# Add the prefix back to matched genes
pv_final_table['Gene'] = pv_final_table['hgnc_symbol'].apply(
    lambda symbol: next((gene for gene in common_genes if gene.endswith(symbol)), None)
)

# Reorganize columns to have prefixed_gene as the first column
# columns_order = ['prefixed_gene'] + [col for col in et_final_table.columns if col != 'prefixed_gene']
columns_order = ['Gene','log2FoldChange','padj','pvalue','baseMean','lfcSE','stat'] #+ [col for col in final_table.columns if col != 'prefixed_gene']

pv_final_table = pv_final_table[columns_order]
pv_final_table = pv_final_table.sort_values(by='padj', ascending=True)


In [ ]:
pv_final_table.to_csv(pv_out,index=False)

In [58]:
mf_final_table = mf_diff[mf_diff['hgnc_symbol'].isin(common_genes_cleaned)].copy()

# Add the prefix back to matched genes
mf_final_table['Gene'] = mf_final_table['hgnc_symbol'].apply(
    lambda symbol: next((gene for gene in common_genes if gene.endswith(symbol)), None)
)

# Reorganize columns to have prefixed_gene as the first column
columns_order = ['Gene','log2FoldChange','padj','pvalue','baseMean','lfcSE','stat'] #+ [col for col in final_table.columns if col != 'prefixed_gene']

mf_final_table = mf_final_table[columns_order]
mf_final_table = mf_final_table.sort_values(by='padj', ascending=True)


In [ ]:
mf_final_table.to_excel(mf_out,index=False)